## Generate predictions on the test data set

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
%matplotlib inline

In [5]:
# the model must be in the models folder
model_to_use = "model-004-0.82.h5"

In [6]:
from keras.models import load_model
model = load_model('models/'+model_to_use)

In [20]:
df = pd.read_csv("original_data/custdatabase.csv")
print('Test dataset shape {0}, {1}'.format(df.shape[0], df.shape[1]))

Test dataset shape 4000, 10


In [21]:
# strip the spaces in the header, if present
df = df.rename(columns=lambda x: x.strip())
df.columns.values

array(['index', 'sex', 'mstatus', 'age', 'children', 'occupation',
       'education', 'income', 'avbal', 'avtrans'], dtype=object)

In [22]:
categorical_features = ["sex", "mstatus", "occupation", "education", "children"]
df[categorical_features].head()
df["children"] = df["children"].astype(str)
label_encoders = {}
label_mappings = {}
for categorical_feature in categorical_features:
    label_encoders[categorical_feature] = preprocessing.LabelEncoder()
    df[categorical_feature + "Num"] = label_encoders[categorical_feature].fit_transform(df[categorical_feature])
    label_mappings[categorical_feature] = label_encoders[categorical_feature].classes_

label_encoders = {}
label_mappings = {}
for categorical_feature in categorical_features:
    label_encoders[categorical_feature] = preprocessing.LabelEncoder()
    df[categorical_feature + "Num"] = label_encoders[categorical_feature].fit_transform(df[categorical_feature])
    label_mappings[categorical_feature] = label_encoders[categorical_feature].classes_

for categorical_feature in categorical_features:
    for class_value in label_mappings[categorical_feature]:
        df[categorical_feature + "_" + (class_value)] = df[categorical_feature] == np.array([(class_value)] * df.shape[0])
        df[categorical_feature + "_" + (class_value)] = df[categorical_feature + "_" + (class_value)].astype(int)

# drop the categorical values
df = df.drop(categorical_features, axis=1)

df.head()

,index,age,income,avbal,avtrans,sexNum,mstatusNum,occupationNum,educationNum,childrenNum,...,occupation_retired,education_postgrad,education_professional,education_secondary,education_tertiary,children_0,children_1,children_2,children_3,children_4
0,1001,44.27,10245.73,53183.04,4069.67,1,0,5,0,2,...,0,1,0,0,0,0,0,1,0,0
1,1002,61.90,1942.57,18100.78,1522.42,1,1,5,2,1,...,0,0,0,1,0,0,1,0,0,0
2,1003,37.30,9896.16,24496.82,2723.35,1,0,0,3,1,...,0,0,0,0,1,0,1,0,0,0
3,1004,25.02,10136.44,26690.01,4143.04,0,0,5,0,1,...,0,1,0,0,0,0,1,0,0,0
4,1005,48.37,2521.11,13439.81,240.26,0,0,4,2,2,...,0,0,0,1,0,0,0,1,0,0


### Remove the "Num" columns only if required

In [23]:
# remove num columns for Neural networks
shouldRemoveNumColumns = True

In [24]:
if shouldRemoveNumColumns:
    num_columns = ["sexNum", "mstatusNum", "occupationNum", "educationNum", "childrenNum"]
    df = df.drop(num_columns, axis=1)
else:
    print("Not dropping the num columns")
df.head()

,index,age,income,avbal,avtrans,sex_F,sex_M,mstatus_divorced,mstatus_married,mstatus_single,...,occupation_retired,education_postgrad,education_professional,education_secondary,education_tertiary,children_0,children_1,children_2,children_3,children_4
0,1001,44.27,10245.73,53183.04,4069.67,0,1,1,0,0,...,0,1,0,0,0,0,0,1,0,0
1,1002,61.90,1942.57,18100.78,1522.42,0,1,0,1,0,...,0,0,0,1,0,0,1,0,0,0
2,1003,37.30,9896.16,24496.82,2723.35,0,1,1,0,0,...,0,0,0,0,1,0,1,0,0,0
3,1004,25.02,10136.44,26690.01,4143.04,1,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
4,1005,48.37,2521.11,13439.81,240.26,1,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
